set params:

In [1]:
import torch
class Args:
    def __init__(self):
        self.dataFile = './dataset/发病率原始数据.xlsx'
        self.gpu = 0  # gpu 卡号
        self.epochs = 200  # 训练轮数
        self.num_layers = 2  # LSTM层数
        self.input_size = 2  # 输入特征的维度
        self.hidden_size = 50  # LSTM节点个数
        self.output_size = 1 # LSTM输出维度
        self.lr = 0.001 #learning rate 学习率
        self.sequence_length = 4  # sequence的长度，默认是用前四个月预测下一个月
        self.batch_size = 8 
        self.useGPU = True  # 默认值
        self.dropout = 0.15
        self.save_file = 'model/LSTM.pth'  # 默认值
        self.device = torch.device("cuda:0")
        
args = Args()  # 创建参数对象


create dataset:


In [2]:
from torch.utils.data import DataLoader, Dataset
class Mydataset(Dataset):
    def __init__(self, xx, yy, transform=None):
        self.x = torch.tensor(xx, dtype=torch.float32)  # 特征数据
        self.y = torch.tensor(yy, dtype=torch.float32).unsqueeze(1)  # 标签数据

    def __getitem__(self, index):
        x1 = self.x[index]  # 获取指定索引的特征数据
        y1 = self.y[index]  # 获取指定索引的标签数据
        return x1, y1  # 否则，直接返回数据

    def __len__(self):
        return len(self.x)  # 返回数据集的长度

In [3]:
from pandas import read_excel
from torch.utils.data import DataLoader, Dataset
import torch
import random
import numpy as np
def getFabingShuData(dataFile, sequence_length, batchSize):   # 单因素预测数据集制作，发病数和发病率都用一个函数
    #fabingshu_data = read_excel('./dataset/发病率原始数据.xlsx')
    fabingshu_data = read_excel(dataFile)
    #fabingshu_data = fabingshu_data['发病率'] # 发病率或是发病数
    fabingshu_data = fabingshu_data.iloc[:,1]
    fabingshu_max = fabingshu_data.max()  # 收盘价的最大值
    fabingshu_min = fabingshu_data.min()  # 收盘价的最小值
    fabingshu_data = (fabingshu_data - min(fabingshu_data)) / (max(fabingshu_data) - min(fabingshu_data))

    #sequence = 4  # 天数
    sequence = sequence_length
    X = []  # 特征
    Y = []  # 标签

    # 遍历数据，创建基于前四个月的数据集
    for i in range(sequence, len(fabingshu_data)):
        temp_x = fabingshu_data[i - sequence:i]
        temp_y = fabingshu_data[i]
        #print(temp_x)
        #print(temp_y)
        X.append(temp_x)
        Y.append(temp_y)
    X = np.array(X, dtype=np.float32)
    Y = np.array(Y, dtype=np.float32)
    # 发病率的时候直接使用原始值，发病数的时候，使用标准化的值
    Y = Y*(fabingshu_max - fabingshu_min)+fabingshu_min   # 原始值

    # 构建训练集和测试集
    total_len = X.shape[0]  # 总数据量

    # print(total_len)
    trainx, trainy = X[:total_len - 30], Y[:total_len - 30]  # 训练集
    index = list(range(len(trainx)))
    random.shuffle(index)

    trainx, trainy = trainx[index[:int(len(index) * 0.90)]], trainy[index[:int(len(index) * 0.90)]]

    testx, testy = X[-30:], Y[-30:]  # 测试集
    #print(trainx.shape, trainy.shape, testx.shape, testy.shape)

    train_loader = DataLoader(dataset=Mydataset(trainx, trainy), batch_size=batchSize,
                              shuffle=True)
    test_loader = DataLoader(dataset=Mydataset(testx, testy), batch_size=batchSize, shuffle=False)  # 测试集数据加载器
    return fabingshu_max, fabingshu_min, train_loader, test_loader  # 返回最大值、最小值、训练集和测试集数据加载器

def getFabingBaiduData(dataFile, sequence_length, batchSize):
    # fabingshu_data = read_excel('./dataset/发病率原始数据.xlsx')
    fabingshu_data = read_excel(dataFile)
    #fabingshu_data = fabingshu_data[['发病率','处理后百度指数']]
    fabingshu_data = fabingshu_data.iloc[:,1:3]

    fabingshu_max = fabingshu_data.iloc[:,0].max()  # 发病最大值
    fabingshu_min = fabingshu_data.iloc[:,0].min()  # 发病最小值
    fabingshu_data = fabingshu_data.apply(lambda x: (x - min(x)) / (max(x) - min(x)))

    #sequence = 4  # 天数
    sequence = sequence_length
    X = []  # 特征
    Y = []  # 标签

    # 遍历数据，创建基于前四个月的数据集
    for i in range(sequence, len(fabingshu_data)):
        temp_x = fabingshu_data.iloc[i - sequence:i, ]
        temp_y = fabingshu_data.iloc[i,0]
        #print(temp_x)
        #print(temp_y)
        X.append(temp_x)
        Y.append(temp_y)
    X = np.array(X, dtype=np.float32)
    Y = np.array(Y, dtype=np.float32)

    # 发病率的时候使用该语句，发病数的时候不适用
    Y = Y*(fabingshu_max - fabingshu_min)+fabingshu_min

    # 构建训练集和测试集
    total_len = X.shape[0]  # 总数据量

    # print(total_len)
    trainx, trainy = X[:total_len - 30], Y[:total_len - 30]  # 训练集
    index = list(range(len(trainx)))
    random.shuffle(index)

    trainx, trainy = trainx[index[:int(len(index) * 0.90)]], trainy[index[:int(len(index) * 0.90)]]

    testx, testy = X[-30:], Y[-30:]  # 测试集
    #print(trainx.shape, trainy.shape, testx.shape, testy.shape)

    train_loader = DataLoader(dataset=Mydataset(trainx, trainy), batch_size=batchSize,
                              shuffle=True)
    test_loader = DataLoader(dataset=Mydataset(testx, testy), batch_size=batchSize, shuffle=False)  # 测试集数据加载器
    return fabingshu_max, fabingshu_min, train_loader, test_loader  # 返回最大值、最小值、训练集和测试集数据加载器

In [4]:
import torch.nn as nn
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        output, _ = self.lstm(x)
        output = self.fc(output[:, -1, :])
        return output

In [5]:
import torch
import numpy as np
import torch.nn as nn
from torch.autograd  import Variable
from LSTMModel import LSTMModel
#from parser_lstm import args

def model_eval(model,test_loader,single_or_daidu,max_num, min_num):
    # 设置模型为评估模式
    model.eval()
    total_loss = 0
    preds = []  # 初始化预测值列表
    labels = []  # 初始化真实标签值列表
    # Evaluate data for one epoch
    for idx, (data, label) in enumerate(test_loader):
        if args.useGPU:
            # 单因素的数据需要升维
            if single_or_daidu == 1:
                data1 = data.unsqueeze(2)  # 删除data张量中的第一个维度，并将其移动到GPU
                pred = model(Variable(data1).cuda())  # 将data1封装成Variable并传入模型进行前向传播，得到预测值
            else:
                pred = model(Variable(data).cuda())  # 将data1封装成Variable并传入模型进行前向传播，得到预测值
            label = label.cuda()  # 将标签数据添加一个维度并移动到GPU
            # print(label.shape)
        else:
            pass
        criterion =nn.MSELoss(reduction='sum')
        loss = criterion(pred, label)
        total_loss += loss.item()
        preds.extend(pred.cpu().detach().numpy().tolist())
        labels.extend(label.cpu().detach().numpy().tolist())

    # # 计算评价指标
    preds = np.array(preds)
    preds = np.squeeze(preds)
    labels = np.array(labels)
    labels = np.squeeze(labels)
    # 如果是发病率，不需要下面两行代码
    #preds = preds * (max_num - min_num) + min_num
    #labels = labels * (max_num - min_num) + min_num

    rmse = np.sqrt(np.mean(np.square(preds - labels)))
    mae = np.mean(np.abs(preds - labels))
    mape = np.mean(np.abs(preds - labels) / labels) * 100
    
    return rmse, mae, mape, preds

In [6]:

import pandas as pd
#from dataset import getFabingShuData,getFabingBaiduData
# 设置参数
#args.datafile = r'./dataset/发病率原始数据.xlsx' # 发病率或是发病数的文件地址
file_path = "model/CV_ST_LSTM_BD_rate/"

single_or_daidu = 2   # 1 表示单因素，2表示双因素，即加百度指数

# 获取测试集的数据
if single_or_daidu==1:
    max_num,min_num,_, test_loader = getFabingShuData(args.dataFile, args.sequence_length, args.batch_size)
elif single_or_daidu==2:
    max_num,min_num, _, test_loader = getFabingBaiduData(args.dataFile, args.sequence_length, args.batch_size)
results = []
evaluations = []
for i in range(10):
    model = torch.load(file_path+"LSTM"+str(i)+".pth")
    rmse, mae, mape, preds = model_eval(model,test_loader,single_or_daidu,max_num, min_num)

    results.append(preds)
    evaluations.append([rmse, mae, mape])
# print results
print('The evaluation results are:')
print(evaluations)
# 求平均值
avgs = np.mean(np.array(evaluations), axis=0)
print('The mean of [rmse, mae, mape] is:')
print(avgs)
# 求方差
vars = np.var(np.array(evaluations), axis=0)
print('The var of [rmse, mae, mape] is:')
print(vars)



The evaluation results are:
[[0.016865462545868765, 0.013494286437829335, 15.171284713319213], [0.01776024447059594, 0.013740424811840058, 15.01444402214734], [0.017426495770192438, 0.01386177676419417, 15.302724541178126], [0.017752236856655064, 0.013809063658118247, 15.185320940175714], [0.01762586330826259, 0.013432793319225311, 14.61483424658088], [0.017090453771292882, 0.013426942378282547, 14.968965404948747], [0.016369600450107862, 0.013361310586333275, 14.998341034903904], [0.017476827260086022, 0.01356358900666237, 14.766410246362502], [0.019528006846250296, 0.014543258026242257, 15.191003069934434], [0.018584202277002766, 0.014120182394981385, 14.922630688916264]]
The mean of [rmse, mae, mape] is:
[1.76479394e-02 1.37353627e-02 1.50135959e+01]
The var of [rmse, mae, mape] is:
[7.07074689e-07 1.23642930e-07 4.00035511e-02]
